In [0]:
 import datetime as dt
 from zoneinfo import ZoneInfo

 from alpaca_pyspark import HistoricalBarsDataSource

 spark.dataSource.register(HistoricalBarsDataSource)

In [0]:
tz = ZoneInfo("America/New_York")

hist_bars_opts = {
    "symbols": ["AAPL", "MSFT", "GOOG"],
    'APCA-API-KEY-ID': dbutils.secrets.get(scope="alpaca_paper_api", key="ALPACA_API_KEY"),
    'APCA-API-SECRET-KEY': dbutils.secrets.get(scope="alpaca_paper_api", key="ALPACA_SECRET_KEY"),
    "timeframe": "1Day",
    "start": dt.datetime(2021, 1, 1, tzinfo=tz).isoformat(),
    "end": dt.datetime(2022, 1, 1, tzinfo=tz).isoformat(),
    "limit": 1000
}

In [0]:
hist_bars_df = (spark.read
                .format("Alpaca_HistoricalBars")
                .options(**hist_bars_opts)
                .load())

In [0]:
display(hist_bars_df)